In [1]:
%connect '/dev/tty.usbserial-14110' --baudrate=115200 --user='micro' --password='M1cr0!!' --wait=0

Connected on /dev/tty.usbserial-14110


In [1]:
import network
import time
from umqtt.robust import MQTTClient
import os
import gc
import sys

In [ ]:
#define AIO_SERVER      "io.adafruit.com"
#define AIO_SERVERPORT  1883                   // use 8883 for SSL
#define AIO_USERNAME  "andrewcgaitskell"
#define AIO_KEY       "aio_dUpf35709LQ92VyZnhRs2w5xekHk"

In [1]:

# create a random MQTT clientID 
random_num = int.from_bytes(os.urandom(3), 'little')
mqtt_client_id = bytes('client_'+str(random_num), 'utf-8')

# connect to Adafruit IO MQTT broker using unsecure TCP (port 1883)
# 
# To use a secure connection (encrypted) with TLS: 
#   set MQTTClient initializer parameter to "ssl=True"
#   Caveat: a secure connection uses about 9k bytes of the heap
#         (about 1/4 of the micropython heap on the ESP8266 platform)
ADAFRUIT_IO_URL = b'io.adafruit.com' 
ADAFRUIT_USERNAME = b'andrewcgaitskell'
ADAFRUIT_IO_KEY = b'aio_dUpf35709LQ92VyZnhRs2w5xekHk'
ADAFRUIT_IO_FEEDNAME = b'esp8266'



In [1]:
client = MQTTClient(client_id=mqtt_client_id, 
                    server=ADAFRUIT_IO_URL, 
                    user=ADAFRUIT_USERNAME, 
                    password=ADAFRUIT_IO_KEY,
                    ssl=False)


In [1]:
try:            
    client.connect()
except Exception as e:
    print('could not connect to MQTT server {}{}'.format(type(e).__name__, e))
    sys.exit()


In [1]:

# publish free heap statistics to Adafruit IO using MQTT
#
# format of feed name:  
#   "ADAFRUIT_USERNAME/feeds/ADAFRUIT_IO_FEEDNAME"
mqtt_feedname = bytes('{:s}/feeds/{:s}'.format(ADAFRUIT_USERNAME, ADAFRUIT_IO_FEEDNAME), 'utf-8')
PUBLISH_PERIOD_IN_SEC = 10 
while True:
    try:
        free_heap_in_bytes = gc.mem_free()
        client.publish(mqtt_feedname,    
                   bytes(str(free_heap_in_bytes), 'utf-8'), 
                   qos=0)  
        time.sleep(PUBLISH_PERIOD_IN_SEC)
    except KeyboardInterrupt:
        print('Ctrl-C pressed...exiting')
        client.disconnect()
        sys.exit()


Comms Exception timeout waiting for first EOF reception